# Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '..', '_pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils
import plot_utils
import interpolation_utils

In [ ]:
plot_utils.set_rcParams()

In [ ]:
fig_num = os.getcwd().split('/')[-1][3:5]
print(fig_num)

# Get data

In [ ]:
cell_data = data_utils.load_var(os.path.join('data', 'cell_data.pkl'))
stim_time = data_utils.load_var(os.path.join('data', 'stim_time.pkl'))
rec_time = data_utils.load_var(os.path.join('data', 'rec_time.pkl'))

In [ ]:
example_stim = data_utils.load_var(os.path.join('data', 'example_stim.pkl'))

# Plot functions

In [ ]:
def plot_uncertainty(time, data, ax, clip=True):
    mu = np.mean(data, axis=0)
    lb = np.mean(data, axis=0) - np.std(data, axis=0)
    ub = np.mean(data, axis=0) + np.std(data, axis=0)
        
    ax.plot(time, mu, c='k', linestyle='--', alpha=0.5, label='mean', linewidth=0.3, clip_on=clip)
    ax.fill_between(time, lb, ub, alpha=0.3, facecolor="k", label='std', clip_on=clip, zorder=10)

In [ ]:
colors = ['C' + str(i) for i in range(10)]
lss = ['-', '--']*3

## Plot optimized stimuli

In [ ]:
def plot_stim(cell_type, ax, idxs=[0], showy=True, showx=True, color=None, rmbspine=True):
        
    for i, idx in enumerate(idxs):
        
        c = color or colors[i]
        ls = lss[i]
    
        ax.plot(
            1e3*stim_time, 1e6*cell_data[cell_type]['best_stimuli'][idx,:], c=c, ls=ls
        )
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    if showy:
        ax.set_ylabel(r'Current ($\mu$A)')
        
    if showx:
        ax.set_xlabel(r'Time (s)')
        ax.spines["bottom"].set_position(("axes", -0.1))
    else:
        if rmbspine:
            ax.spines['bottom'].set_visible(False)
            ax.set_xticks([])
        
    ax.set_ylim(-0.52, 0.52)

In [ ]:
idxs_ON = [0,1]

In [ ]:
plot_stim('ON', idxs=idxs_ON, ax=plt.subplot(111))

In [ ]:
idxs_OFF = [0,1]

In [ ]:
plot_stim('OFF', idxs=idxs_OFF, ax=plt.subplot(111))

##### Export data

In [ ]:
data_utils.make_dir('source_data')

In [ ]:
stimuli_exdf = pd.DataFrame()
stimuli_exdf['Time/ms'] = 1e3*stim_time

for idx in idxs_OFF:
    stimuli_exdf['OFF_Current' + str(idx+1) + '/uA'] = 1e6*cell_data['OFF']['best_stimuli'][idx,:]
    
for idx in idxs_ON:
    stimuli_exdf['ON_Current' + str(idx+1) + '/uA'] = 1e6*cell_data['ON']['best_stimuli'][idx,:]

stimuli_exdf.to_csv('source_data/Optimized_currents.csv', float_format='%.4f', index=False)

stimuli_exdf = pd.read_csv('source_data/Optimized_currents.csv')
stimuli_exdf.plot(x='Time/ms')

## Plot stimulus example.

In [ ]:
p_N = example_stim['anchor_points'].size - 3

In [ ]:
def plot_stim_param(ax, showy=True, showx=True):
        
    ax.plot(
        1e3*example_stim['anchor_time'],
        1e6*example_stim['anchor_points'],
        marker='o', ls='None', c='k', ms='3', clip_on=False
    )

    for i, (x, y) in enumerate(zip(example_stim['anchor_time'], example_stim['anchor_points'])):
        if i == 0 or i == p_N+2:
            ax.text(1e3*x, 1e6*y, r'  $0$')
        elif i == p_N+1:
            ax.text(1e3*x, 1e6*y, r'  $p^*$')
        else:
            ax.text(1e3*x, 1e6*y, r'  $p_' + str(i)+ '$')
    
    ax.plot(1e3*example_stim['time'], 1e6*example_stim['stim'], color='dimgray', clip_on=False)
    ax.fill_between(1e3*example_stim['time'], 1e6*example_stim['stim'], alpha=0.3, color='dimgray', clip_on=False, lw=0)
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    if showy:
        ax.set_ylabel(r'Current ($\mu$A)')
        
    else:
        ax.spines['left'].set_visible(False)
        
    if showx:
        ax.set_xlabel(r'Time (ms)')
        ax.spines["bottom"].set_position(("axes", -0.1))
    else:
        ax.spines['bottom'].set_visible(False)
        ax.set_xticks([])
        
    ax.set_ylim(-0.5, 0.5)

In [ ]:
plot_stim_param(ax=plt.subplot(111), showx=False)

## Plot response

In [ ]:
def get_response_sorted(cell_type_opt, cell_type_show):
    
    if cell_type_show == 'ON': cell_type_show = 'CBC5o'
    if cell_type_show == 'OFF': cell_type_show = 'CBC3a'
    
    d_sort_idx = np.argsort(cell_data[cell_type_opt]['samples']['loss']['total'])
    rates_sorted = cell_data[cell_type_opt]['samples'][cell_type_show]['rate'][d_sort_idx]
    
    return rates_sorted

def plot_response(cell_type_opt, cell_type_show, ax, idxs=[0], showx=True, showy=True, cumrel=True):

    rrps = cell_data[cell_type_show]['rrps']
    rates_sorted = get_response_sorted(cell_type_opt, cell_type_show)
    
    if cumrel:
        rates_sorted = np.cumsum(rates_sorted*np.mean(np.diff(rec_time)), axis=1) / rrps
    
    for i, idx in enumerate(idxs):
        color = colors[i]
        ls = lss[i]
        
        ax.plot(1e3*rec_time, np.mean(rates_sorted[idx,:], axis=-1), color=color, clip_on=False, ls=ls)
        
        ax.fill_between(
            1e3*rec_time,
            np.min(rates_sorted[idx,:], axis=-1),
            np.max(rates_sorted[idx,:], axis=-1),
            alpha=0.4, color=color, lw=0.0, clip_on=False,
        )
        
    if showy:
        if cumrel:
            ax.set_ylabel(r'Cumulative' + '\n' + 'release / $v^{max}_{RRP}$')
            ax.set_yticks([0,1])
        else:
            ax.set_ylabel(r'Rate (ves./s)')
    else:
        ax.spines['left'].set_visible(False)
        ax.set_yticks([])
        
    if showx:
        ax.set_xlabel(r'Time (ms)')
        ax.spines["bottom"].set_position(("axes", -0.1))
    else:
        ax.spines['bottom'].set_visible(False)
        ax.set_xticks([])
        
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    ax.set_ylim(0, rate_ylim)

In [ ]:
rate_ylim = 1

In [ ]:
plot_response('ON', 'ON', ax=plt.subplot(111), idxs=idxs_ON, showx=False, showy=True)

##### Export data

In [ ]:
for cell_type_opt in ['ON', 'OFF']:
    if cell_type_opt == 'ON': stim_idxs = idxs_ON
    if cell_type_opt == 'OFF': stim_idxs = idxs_OFF
        
    for stim_idx in stim_idxs:
        for cell_type_show in ['ON', 'OFF']:
            rates_sorted = get_response_sorted(cell_type_opt, cell_type_show)

            response_exdf = pd.DataFrame()
            response_exdf['Time/ms'] = 1e3*rec_time

            for cell_idx in range(rates_sorted.shape[-1]):
                response_exdf[cell_type_show + ' cell' + str(cell_idx+1) + ': Release/(ves/s)'] =\
                    rates_sorted[stim_idx,:,cell_idx]

            filename = 'source_data/' + cell_type_opt + '_Current' + str(stim_idx+1) + '_' + cell_type_show + '_Release.csv'
            print(filename)
            response_exdf.to_csv(filename, float_format='%.4f', index=False)

In [ ]:
for cell_type_opt in ['ON', 'OFF']:
    if cell_type_opt == 'ON': stim_idxs = idxs_ON
    if cell_type_opt == 'OFF': stim_idxs = idxs_OFF
        
    fig, axs = plt.subplots(1,4,figsize=(12,3), sharey=True)
        
    i = 0
    for stim_idx in stim_idxs:
        for cell_type_show in ['ON', 'OFF']:
            filename = 'source_data/' + cell_type_opt + '_Current' + str(stim_idx+1) + '_' + cell_type_show + '_Release.csv'
            response_exdf = pd.read_csv(filename)
            axs[i].set_title(cell_type_opt + 'stim/response' + cell_type_show)
            rrps = cell_data[cell_type_show]['rrps']
            axs[i].plot(response_exdf['Time/ms'],
                        np.cumsum(response_exdf.iloc[:,1:]*np.mean(np.diff(rec_time)), axis=0) / rrps)
            i += 1
    plt.show()

In [ ]:
pd.DataFrame({'ON': cell_data['ON']['rrps'], 'OFF': cell_data['OFF']['rrps']},
             index=['Cell'+str(i+1) for i in range(5)]).to_csv('source_data/RRP_sizes.csv', float_format="%.4f")

## Plot Vext

In [ ]:
scale_factor = 1e6
Vm_scale_factor = 1e3

In [ ]:
def plot_Vext(ax, Vext_file, retina_h=105, ames_h=35, plot_r=None, el_r=None):
    
    Vext = pd.read_csv(Vext_file, comment="%", header=None, names=['x', 'r', 'V'])
    Vim  = interpolation_utils.interpolate_xyz2grid(Vext['x'], Vext['r'], Vext['V'])[2]
    Vim  = np.concatenate([np.fliplr(Vim),Vim[:,1:]], axis=1)*Vm_scale_factor # Mirror.
    
    ax.axis('off')
    
    w = Vext['r'].max()*scale_factor
    h = Vext['x'].max()*scale_factor
    
    heatmap = ax.imshow(
        Vim, origin='lower', cmap='Reds', extent=(-w, w, 0, h),
        vmin=0, interpolation='bicubic', vmax=100
    )
    
    if el_r is not None:
        ax.fill_between([-el_r, el_r], [-2,-2], [0, 0], color='k', lw=0, clip_on=False)
    
    ax.text(-plot_r+5, retina_h+35/2, 'Ames\' medium', verticalalignment='center', c='gray')
    ax.text(-plot_r+5, retina_h-35/2, 'Retina', verticalalignment='center', c='gray')
    
    ax.plot([30, 30], [25, 75], c='k')
    ax.text(40, 50, r"50 $\mu m$", va='center', ha='left')
    
    if plot_r is not None:
        ax.set_xlim((-plot_r, plot_r))
    else:
        ax.set_xlim(-w, w)
    ax.set_ylim(0, retina_h + ames_h)
    
    ax.axhline(retina_h, c='k', linestyle='--')        
  
    ytop = 27
    ybot = 6
    ax.plot([-28, -28], [ybot, ytop], c='dimgray')
    ax.text(-32, ybot+(ytop-ybot)*0.5, r"Dendrites", va='center', ha='right')
        
    ytop = 65
    ybot = 36
    ax.plot([-28, -28], [ybot, ytop], c='dimgray')
    ax.text(-32, ybot+(ytop-ybot)*0.5, r"Axon", va='center', ha='right')

In [ ]:
def plot_comps(ax, comps_file, scale_comps):
    
    cell_comps = pd.read_csv(comps_file, names=['x', 'y', 'z'], delim_whitespace=True)
    
    ax.scatter(
        cell_comps['x']*scale_factor, cell_comps['z']*scale_factor,
        marker='o', color="k", alpha=1, s=scale_comps, facecolor='white', zorder=20, lw=scale_comps/5
    )
    ax.scatter(
        cell_comps['x'][0]*scale_factor, cell_comps['z'][0]*scale_factor,
        marker='o', color="k", alpha=1, s=scale_comps*4, facecolor='white', zorder=20, lw=scale_comps/5
    )
    
    zsoma = cell_comps.loc[0]['z']
    
    relzs = cell_comps['z'].values - zsoma
    
    nextcomp1 = np.argmin(relzs + (relzs<=0)*1000)
    nextcomp2 = np.argmax(relzs + (relzs>=0)*-1000)
    
    def plot_line(comp1, comp2):
        ax.plot(
            np.array([cell_comps['x'][comp1], cell_comps['x'][comp2]])*scale_factor,
            np.array([cell_comps['z'][comp1], cell_comps['z'][comp2]])*scale_factor,
            color="dimgray", alpha=1, lw=scale_comps/2
        )
    
    plot_line(comp1=0, comp2=nextcomp1)
    plot_line(comp1=0, comp2=nextcomp2)

In [ ]:
comps_file = os.path.join('..', '..', 'COMSOL2retsim_interface', 'comsol_input', 'global', 'ON.csv')
cell_comps = pd.read_csv(comps_file, names=['x', 'y', 'z'], delim_whitespace=True)

In [ ]:
def plot_Vext_and_comps(ax, cell, scale_comps=8, plot_r=105):

    Vext_file = os.path.join('..', 'gen_Vext', 'Vext_simple.csv')
    comps_file = os.path.join('..', '..', 'COMSOL2retsim_interface', 'comsol_input', 'global', f'{cell}.csv')

    assert os.path.isfile(comps_file)

    plot_Vext(ax, Vext_file, retina_h=105, plot_r=plot_r, el_r=15)
    plot_comps(ax, comps_file, scale_comps=scale_comps)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(3,2))
plot_Vext_and_comps(ax=ax, cell='ON', scale_comps=2)

# Make paper figure

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(5.6,2.7), gridspec_kw=dict(height_ratios=[1,1,1]))

# Stimulus column.
axs[0,0].set_title('A              ', loc='left', ha='right', fontweight="bold")
axs[1,0].set_title('C              ', loc='left', ha='right', fontweight="bold")
axs[2,0].set_title('E              ', loc='left', ha='right', fontweight="bold")

plot_stim_param(ax=axs[0,0], showx=False)
plot_stim(ax=axs[1,0], cell_type='OFF', showx=False, idxs=idxs_OFF)
plot_stim(ax=axs[2,0], cell_type='ON', idxs=idxs_ON)

# Vext row.
axs[0,1].set_title('B i            ', loc='left', ha='right', fontweight="bold")
axs[0,1].set_title('OFF', loc='center', ha='center')
axs[0,1].axis('off')

axs[0,2].set_title('B ii ', loc='left', ha='right', fontweight="bold")
axs[0,2].set_title('ON', loc='center', ha='center')
axs[0,2].axis('off')

# Response to OFF stimulus.
rate_ylim = 1

axs[1,1].set_title('D i            ', loc='left', ha='right', fontweight="bold")
plot_response(ax=axs[1,1], cell_type_opt='OFF', cell_type_show='OFF', showx=False, showy=True, idxs=idxs_OFF)

axs[1,2].set_title('D ii ', loc='left', ha='right', fontweight="bold")
plot_response(ax=axs[1,2], cell_type_opt='OFF', cell_type_show='ON', showx=False, showy=False, idxs=idxs_OFF)

# Response to ON stimulus.
rate_ylim = 1

axs[2,1].set_title('F i            ', loc='left', ha='right', fontweight="bold")
plot_response(ax=axs[2,1], cell_type_opt='ON', cell_type_show='OFF', showx=True, showy=True, idxs=idxs_ON)

axs[2,2].set_title('F ii ', loc='left', ha='right', fontweight="bold")
plot_response(ax=axs[2,2], cell_type_opt='ON', cell_type_show='ON', showx=True, showy=False, idxs=idxs_ON)

fig.align_ylabels(axs)
plt.tight_layout(w_pad=-0.2, h_pad=0.1, rect=[-0.02,-0.03,1.02,1.04])

for ax in axs[:,1]:
    ax.set_position(np.array(ax.get_position().bounds) + [0.04, 0,0,0])

# Plot Vext.
plot_Vext_and_comps(ax=fig.add_axes(axs[0,1].get_position().bounds), cell='OFF', scale_comps=0.6, plot_r=150)
plot_Vext_and_comps(ax=fig.add_axes(axs[0,2].get_position().bounds), cell='ON', scale_comps=0.6, plot_r=150)
    
plt.savefig(f'../_figures/fig{fig_num}_optimized_stimuli.pdf')
plt.show()

# Make appendix figure

In [ ]:
os.listdir('removed_ion_channels')

In [ ]:
cell_colors = ['darkgreen', 'steelblue']
cell_lss = ['-', (0, (5,2))]

In [ ]:
def extract_data_list(rec_type, rec_data_cell, cum_rate=True):

    if rec_type == 'rate':
        cols = [col for col in rec_data_cell[0]['Data'].columns if 'rate' in col]

    elif rec_type == 'Vext':
        cols_V = []
        cols_Vm = []
        for col in rec_data_cell[0]['Data'].columns:
            if 'rate' in col:
                node = int(col[col.find('(')+1:col.find(')')])
                cols_V.append('V (' + str(node) + ')')
                cols_Vm.append('Vm (' + str(node) + ')')

    else:
        cols = []
        for col in rec_data_cell[0]['Data'].columns:
            if 'rate' in col:
                node = int(col[col.find('(')+1:col.find(')')])
                cols.append(rec_type + ' (' + str(node) + ')')

    cell_type = 'ON' if rec_data_cell[0]['cell'] == 'CBC5o' else 'OFF'
        
    if rec_type == 'Ca':
        unit_factor = 1e6
    elif 'V' in rec_type:
        unit_factor = 1e3
    else:
        unit_factor = 1
        
    data_list = []
        
    for idx, rec_data in enumerate(rec_data_cell):
        if rec_type == 'Vext':
            data = rec_data['Data'].loc[:,cols_V].values - rec_data['Data'].loc[:,cols_Vm].values
        else:
            data = rec_data['Data'].loc[:,cols].values
        
        if rec_type == 'rate' and cum_rate:
            data = np.cumsum(data, axis=0)*(rec_time[1] - rec_time[0]) / cell_data[cell_type]['rrps'][idx]
            
        data_list.append(data*unit_factor)
        
    return data_list, cell_type

In [ ]:
N_param_sets = 5

def plot_synapses_mean_and_uncertainty(ax, rec_data_stim, rec_type, rm_offset, mean_kw={"lw": 0.8}):
    
    assert len(rec_data_stim) == 2*N_param_sets
    
    for idx, rec_data_cell in enumerate([rec_data_stim[N_param_sets:], rec_data_stim[:N_param_sets]]):

        color = cell_colors[idx]
        ls = cell_lss[idx]
        
        data_list, cell_type = extract_data_list(rec_type, rec_data_cell)
        
        for idx, data in enumerate(data_list):         

            mu = np.mean(data, axis=1)
            if rm_offset: mu -= mu[0]

            ax.plot(rec_time*1e3, mu, **mean_kw, color=color, ls=ls, clip_on=False, label=cell_type)
            
            if rec_type == 'Vext': break

In [ ]:
ABC = 'ABCDEFGHIJK'

def set_titles(ax_row, rowi):
    ax_row[0].set_title(ABC[rowi] + '                  ', loc='left', fontweight="bold", ha='right', va='top')

In [ ]:
import seaborn as sns

sbnx = 3
sbny = 7

height_ratios = np.full(sbny, 1)
height_ratios[3] = 0.5

fig, axs = plt.subplots(sbny, sbnx, figsize=(5.6,sbny*0.75), squeeze=False, sharey='row', sharex=True,
                        gridspec_kw=dict(height_ratios=height_ratios))

rowi = 0

ax_row = axs[rowi, :]; set_titles(ax_row, rowi); rowi += 1;
plot_stim('ON',  ax_row[0], idxs=[0], showy=True, showx=False, color='k', rmbspine=False)
plot_stim('ON',  ax_row[1], idxs=[1], showy=False, showx=False, color='k', rmbspine=False)
plot_stim('OFF', ax_row[2], idxs=[0], showy=False, showx=False, color='k', rmbspine=False)

# All channels
rec_data_modified = data_utils.load_var(os.path.join('removed_ion_channels', 'all_params.pkl'))[1]

ax_row = axs[rowi, :]; set_titles(ax_row, rowi); rowi += 1;
ax_row[0].set_ylabel(r'V$_\mathrm{ex}$ (mV)')
ax_row[0].set_ylim(-10, 10)
plot_synapses_mean_and_uncertainty(ax_row[0], rec_data_modified[0], rec_type='Vext', rm_offset=False)
plot_synapses_mean_and_uncertainty(ax_row[1], rec_data_modified[1], rec_type='Vext', rm_offset=False)
plot_synapses_mean_and_uncertainty(ax_row[2], rec_data_modified[2], rec_type='Vext', rm_offset=False)

ax_row[-1].legend(
    handlelength=1.4,
    loc='upper right', borderaxespad=0., labelspacing=0.1, frameon=False, #bbox_to_anchor=(0, 1)
)

ax_row = axs[rowi, :]; set_titles(ax_row, rowi); rowi += 1;
ax_row[0].set_ylabel(r'$\Delta$ V$_\mathrm{m}$ (mV)')
ax_row[0].set_ylim(-15, 15)
plot_synapses_mean_and_uncertainty(ax_row[0], rec_data_modified[0], rec_type='Vm', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[1], rec_data_modified[1], rec_type='Vm', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[2], rec_data_modified[2], rec_type='Vm', rm_offset=True)

# Some space.

ax_row = axs[rowi, :]; rowi += 1;
for ax in ax_row: ax.axis('off')

ax_row = axs[rowi, :]; set_titles(ax_row, rowi-1); rowi += 1;
ax_row[0].set_ylabel(r'Cumulative' + '\n' + 'release / $v^{max}_{RRP}$')
ax_row[0].set_yticks([0,1])
ax_row[0].set_ylim([0,1.1])
plot_synapses_mean_and_uncertainty(ax_row[0], rec_data_modified[0], rec_type='rate', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[1], rec_data_modified[1], rec_type='rate', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[2], rec_data_modified[2], rec_type='rate', rm_offset=True)

# Without T-Type channel
rec_data_modified = data_utils.load_var(os.path.join('removed_ion_channels', 'rm_T_at.pkl'))[1]

ax_row = axs[rowi, :]; set_titles(ax_row, rowi-1); rowi += 1;
ax_row[0].set_ylabel(r'Cumulative' + '\n' + 'release / $v^{max}_{RRP}$')
ax_row[0].set_yticks([0,1])
ax_row[0].set_ylim([0,1.1])
plot_synapses_mean_and_uncertainty(ax_row[0], rec_data_modified[0], rec_type='rate', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[1], rec_data_modified[1], rec_type='rate', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[2], rec_data_modified[2], rec_type='rate', rm_offset=True)

# Without L-Type channel for OFF cell.
rec_data_modified = data_utils.load_var(os.path.join('removed_ion_channels', 'rm_L_at.pkl'))[1]

ax_row = axs[rowi, :]; set_titles(ax_row, rowi-1); rowi += 1;
ax_row[0].set_ylabel(r'Cumulative' + '\n' + 'release / $v^{max}_{RRP}$')
ax_row[0].set_yticks([0,1])
ax_row[0].set_ylim([0,1.1])
plot_synapses_mean_and_uncertainty(ax_row[0], rec_data_modified[0], rec_type='rate', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[1], rec_data_modified[1], rec_type='rate', rm_offset=True)
plot_synapses_mean_and_uncertainty(ax_row[2], rec_data_modified[2], rec_type='rate', rm_offset=True)

for ax in ax_row: ax.set_xlabel('Time (ms)')

sns.despine()
fig.align_ylabels()
plt.tight_layout(h_pad=0.1)

plt.savefig(f'../_figures_apx/figapx{fig_num}_optimized_stimuli_rm_channels.pdf')
plt.show()

##### Export

In [ ]:
for cell_type_opt in ['ON', 'OFF']:
    if cell_type_opt == 'ON':
        stim_idxs = [0,1]
    if cell_type_opt == 'OFF':
        stim_idxs = [2]
        
    for ca_type in ['T', 'L']:
        rec_data_modified = data_utils.load_var(os.path.join('removed_ion_channels', 'rm_' + ca_type + '_at.pkl'))[1]
        
        for stim_idx in stim_idxs:
            response_exdf = pd.DataFrame()
            response_exdf['Time/ms'] = 1e3*rec_time

            rate_list, _ = extract_data_list(
                'rate', rec_data_modified[stim_idx][5:], cum_rate=False)
            
            for cell_idx, rate in enumerate(rate_list):
                response_exdf['OFF' + ' cell' + str(cell_idx+1) + ': Release/(ves./s)'] = rate.mean(axis=1)

            filename = f'source_data/cell_type_opt_Current{stim_idx%2+1}_OFF_Release_rm_Ca_{ca_type}.csv'
            print(filename)
            response_exdf.to_csv(filename, float_format='%.4f', index=False)

##### Test

In [ ]:
for ca_type in ['T', 'L']:   
    fig, axs = plt.subplots(1,3,figsize=(12,3), sharey=True, subplot_kw=dict(ylim=(0,1.2)))
        
    filenames = [filename for filename in os.listdir('source_data') if 'rm_Ca_'+ca_type in filename]
        
    for ax, filename in zip(axs, np.flip(filenames)):
        response_exdf = pd.read_csv('source_data/' + filename)
        ax.set_title(filename, fontsize=8)
        ax.plot(response_exdf['Time/ms'],
                np.cumsum(response_exdf.iloc[:,1:]*np.mean(np.diff(rec_time)), axis=0) / cell_data['OFF']['rrps'],
                c='green')
    plt.show()

##### Export

In [ ]:
rec_data_modified = data_utils.load_var(os.path.join('removed_ion_channels', 'all_params.pkl'))[1]

for rec_data_stim, (stim_idx, cell_type_opt) in zip(rec_data_modified, [(0, 'ON'), (1, 'ON'), (0, 'OFF')]):
    for cell_type_show in ['ON', 'OFF']:
       
        cell_idxs = (0,5) if cell_type_show == 'ON' else (5,10)

        response_exdf = pd.DataFrame()
        response_exdf['Time/ms'] = 1e3*rec_time

        Vex_list, _ = extract_data_list(
            'Vext', rec_data_stim[cell_idxs[0]:cell_idxs[1]])

        for comp_idx, Vext in enumerate(Vex_list[0].T):
            response_exdf[f'{cell_type_show} compartment{comp_idx+1}: Extracellular voltage/mV'] = Vext

        Vm_list, _ = extract_data_list(
            'Vm', rec_data_modified[stim_idx][cell_idxs[0]:cell_idxs[1]])

        for cell_idx, Vm_cell in enumerate(Vm_list):
            for comp_idx, Vm in enumerate(Vm_cell.T):
                response_exdf[f'{cell_type_show} cell{cell_idx+1} compartment{comp_idx+1}: Membrane voltage/mV'] = Vext

        filename = f'source_data/{cell_type_opt}_Current{stim_idx+1}_{cell_type_show}_Vext_and_Vm.csv'
        print(filename)
        response_exdf.to_csv(filename, float_format='%.4f', index=False)

##### Test

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(12,3))

for ax_col, (stim_idx, cell_type_opt) in zip(axs.T, [(0, 'ON'), (1, 'ON'), (0, 'OFF')]):   
    for cell_type_show in ['ON', 'OFF']:

        color = 'b' if cell_type_show == 'ON' else 'green'

        filename = f'source_data/{cell_type_opt}_Current{stim_idx+1}_{cell_type_show}_Vext_and_Vm.csv'
        
        df = pd.read_csv(filename)
        ax_col[0].plot(df['Time/ms'], df[[col for col in df.columns if 'Ex' in col]].mean(axis=1),
                       c=color, label=cell_type_show)
        ax_col[0].set_title(filename.split('/')[-1])
        ax_col[0].legend()
        
        ax_col[1].plot(df['Time/ms'], df[[col for col in df.columns if 'Mem' in col]].mean(axis=1),
                       c=color, label=cell_type_show)
        ax_col[1].set_title(filename.split('/')[-1])
        ax_col[1].legend()
plt.tight_layout()